In [4]:
!pip install langchain langchain-community langchain-openai langchain-gemini faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableLambda

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install langchain-google-genai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [16]:
!pip install dateparser dotenv langchain_google_genai pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 5.2 MB/s eta 0:00:00


### Make .env file and and add GOOGLE_API_KEY  
https://aistudio.google.com/apikey

# FINAL MODEL

In [21]:
import pandas as pd
import re
from datetime import datetime
import dateparser
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores.faiss import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain

from langchain.agents import Tool, create_react_agent, AgentExecutor
from langchain import hub

load_dotenv()
vector_store = None
appointment_df = pd.DataFrame(columns=['Name', 'Phone', 'Email', 'Date', 'Message'])

def get_vector_store():
    global vector_store
    if vector_store is None:
        try:
            # Load PDF files
            pdf_loader = DirectoryLoader('/content/', glob='*.pdf', loader_cls=PyPDFLoader)
            pdf_documents = pdf_loader.load()

            # Load TXT files
            txt_loader = DirectoryLoader('/content/', glob='*.txt', loader_cls=TextLoader)
            txt_documents = txt_loader.load()

            documents = pdf_documents + txt_documents

            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
            texts = text_splitter.split_documents(documents)

            embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

            vector_store = FAISS.from_documents(texts, embedding=embeddings)
            print("Vector store initialized successfully.")
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            return None
    return vector_store

def answer_from_documents(query: str) -> str:
    vs = get_vector_store()
    if not vs:
        return "Sorry, the document system is not available right now."

    docs = vs.similarity_search(query, k=3)

    prompt_template = """
    Answer the question as detailed as possible from the provided context. Make sure to provide all the details.
    If the answer is not in the provided context, just say, "The answer is not available in the context".
    Don't provide a wrong answer.

    Context:\n{context}\n
    Question:\n{question}\n
    Answer:
    """
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3)
    chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)

    response = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
    return response["output_text"]


class AppointmentState:
    def __init__(self):
        self.reset()

    def reset(self):
        self.stage = "initial"
        self.name = None
        self.phone = None
        self.email = None
        self.date = None
        self.message = None

    def is_complete(self):
        return self.name and self.phone and self.email and self.date and self.message

appointment_state = AppointmentState()

def is_valid_phone(phone: str) -> bool:
    pattern = re.compile(r"^\+?1?\d{9,15}$")
    return pattern.match(phone) is not None

def is_valid_email(email: str) -> bool:
    pattern = re.compile(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$")
    return pattern.match(email) is not None

def parse_date(date_string: str) -> str:
    try:
        parsed_date = dateparser.parse(date_string)
        if parsed_date:
            return parsed_date.strftime('%Y-%m-%d')
        return None
    except:
        return None

def save_appointments_to_file(df: pd.DataFrame, filepath: str):
    """Saves the appointment DataFrame to a CSV file.

    Args:
        df: The DataFrame containing appointment details.
        filepath: The path to save the CSV file.
    """
    df.to_csv(filepath, index=False)
    print(f"Appointment data saved to {filepath}")


def book_appointment(user_input: str) -> str:
    state = appointment_state
    global appointment_df

    if state.stage == "initial":
        parsed_date = parse_date(user_input)
        if parsed_date:
            state.date = parsed_date
            state.stage = "awaiting_name"
            return f"Sure, I can schedule a call for {state.date}. What is your full name?"
        else:
            state.stage = "awaiting_date"
            return "Of course. What date would you like to schedule the call for?"

    if state.stage == "awaiting_date":
        parsed_date = parse_date(user_input)
        if parsed_date:
            state.date = parsed_date
            state.stage = "awaiting_name"
            return f"Great, I have the date as {state.date}. What is your full name?"
        else:
            return "I'm sorry, I couldn't understand that date. Please provide a clear date like 'tomorrow', 'next Friday', or '2024-08-15'."

    if state.stage == "awaiting_name":
        state.name = user_input
        state.stage = "awaiting_phone"
        return f"Thanks, {state.name}. What is your phone number?"

    if state.stage == "awaiting_phone":
        if is_valid_phone(user_input):
            state.phone = user_input
            state.stage = "awaiting_email"
            return "Got it. And finally, what is your email address?"
        else:
            return "That doesn't seem like a valid phone number. Please enter a valid phone number (e.g., +1234567890)."

    if state.stage == "awaiting_email":
        if is_valid_email(user_input):
            state.email = user_input
            state.stage = "awaiting_message"
            return "Finally what is the message for the user? "
        else:
            return "That email address appears to be invalid. Please provide a valid email."

    if state.stage == "awaiting_message":
        state.message = user_input
        if state.is_complete():
            confirmation_message = (
                f"Thank you! Your appointment is confirmed:\n"
                f"Name: {state.name}\n"
                f"Phone: {state.phone}\n"
                f"Email: {state.email}\n"
                f"Date: {state.date}\n"
                f"Message: {state.message}\n"
                "Someone from our team will contact you."
            )
            # Append to DataFrame
            new_appointment = pd.DataFrame([{
                'Name': state.name,
                'Phone': state.phone,
                'Email': state.email,
                'Date': state.date,
                'Message': state.message
            }])
            appointment_df = pd.concat([appointment_df, new_appointment], ignore_index=True)

            # Save to file
            save_appointments_to_file(appointment_df, "appointments.csv")

            state.reset()

            return confirmation_message

    return "I'm sorry, I'm a bit confused. Could you please start the booking process again?"


def create_agent_executor():
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

    tools = [
        Tool(
            name="DocumentQA",
            func=answer_from_documents,
            description="""
                          Use this tool to answer questions only from the documents if the documents are provided"""
        ),
        Tool(
            name="AppointmentScheduler",
            func=book_appointment,
            description="""
                        Use this tool when a user asks to schedule an appointment, book a call,
                        or wants someone to contact them. This tool will conversationally
                        collect the necessary information (name, phone, email, date).
                        Pass the user's entire request as input.
                        """,
        ),
    ]

    prompt = hub.pull("hwchase17/react-chat")

    agent = create_react_agent(llm, tools, prompt)

    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

    return agent_executor


if __name__ == "__main__":
    agent_executor = create_agent_executor()

    print("Chatbot is ready! Ask me questions about the documents or ask me to book for appointment.")

    chat_history = []

    while True:
        user_input = input("You : ")
        if user_input.lower() in ["exit", "quit","thank you"]:
            print("Goodbye!")
            break

        if appointment_state.stage != 'initial':
            response = book_appointment(user_input)
        else:
            result = agent_executor.invoke({
                "input": user_input,
                "chat_history": chat_history
            })
            response = result['output']

        print(f"Chatbot: {response}")

        chat_history.append(f"User: {user_input}")
        chat_history.append(f"AI: {response}")


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Chatbot is ready! Ask me questions about the documents or ask me to book for appointment.
You : what is the document about?


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: DocumentQA
Action Input: what is the document about?Vector store initialized successfully.
The document is about a model that uses an encoder to map an input sequence of symbol representations to a sequence of continuous representations. A decoder then generates an output sequence of symbols one element at a time, consuming previously generated symbols as additional input. The document also describes an attention function that maps a query and a set of key-value pairs to an output. It also mentions that self-attention layers are faster than recurrent layers in terms of computational complexity when the sequence.Do I need to use a tool? No
Final Answer: The document describes a model that uses an encoder-decoder structure with an attention mechanism. The encoder maps an input 

# DOWNLOAD LLM AND RUN LOCALLY

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

In [ ]:
from langchain.schema.runnable import RunnableLambda
from langchain_core.messages import HumanMessage, AIMessage ,SystemMessage
import torch

def LLM(messages):
    prompt = messages
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=512,
            temperature=0.6,
            do_sample=True,
            top_p=0.8,
            eos_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Wrap in LangChain Runnable
llm = RunnableLambda(lambda messages: LLM(messages))

In [ ]:
llm.invoke(final_prompt)